In [91]:
import sqlalchemy as sa
import pandas as pd
import pyarrow as pa
import credentials_db as creds

In [3]:
pd.options.mode.copy_on_write = True

In [4]:
engine_sda = sa.create_engine('oracle+cx_oracle://'+creds.sda['usr']+':'+creds.sda['pwd']+creds.sda['host']+'/'+creds.sda['db'])

In [54]:
sql_enrolllment_data_fall_variable = open('./sql/enrollment data fall census.sql','r').read()

In [56]:
queries = []
for i in range (19,25):
    queries.append(sql_enrolllment_data_fall_variable.replace('__yr__',str(i)))
sql_enrolllment_data_fall = '\nunion\n'.join(queries)

In [57]:
enrollment = pd.read_sql(sql_enrolllment_data_fall,engine_sda,dtype={
        'sytdean_pidm':'uint32[pyarrow]',
        'sytdean_term_code_key':'category',
        'sytdean_registered_ind':'category',
        'sytdean_camp_code':'category',
        'sytdean_levl_code':'category',
        'sytdean_coll_code_1':'category',
        'sytdean_majr_code_1':'category',
        'sytdean_styp_code':'category',
        'sytdean_class_code':'category'
    })
    